In [9]:
import numpy as np
import pandas as pd
import random
from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis


def build_confusion_matrix(dataset, model):
        X_train, X_test, y_train, y_test = train_test_split(dataset.iloc[:, :-1], dataset.iloc[:, -1], test_size=0.25)
        predictions = model.fit(X_train, y_train).predict(X_test)
        tn, fn, fp, tp = confusion_matrix(predictions, y_test.values).ravel()
        return {'true_negative': tn, 'false_negative': fn, 'false_positive': fp, 'true_positive': tp}

def sensitivity(cm):
    return cm['true_positive'] / (cm['true_positive'] + cm['false_negative'])

def specificity(cm):
    return cm['true_negative'] / ( cm['true_negative'] + cm['false_positive'] )

def false_positive_rate(cm):
    return (1.0 - (specificity(cm)) )

def generate_scores(dataset, model):
    specificity_scores = []
    sensitivity_scores = []

    for n in range(1000):
        cm = build_confusion_matrix(dataset, model)
        specificity_scores.append(specificity(cm))
        sensitivity_scores.append(sensitivity(cm))
    
    spec_avg = pd.Series(specificity_scores).mean()
    sens_avg = pd.Series(sensitivity_scores).mean()
    
    return {'specificity': spec_avg, 'sensitivity': sens_avg }

def summarise_test(test):
    x ='For {}, in model {}\nAverage NO SIGFALL accuracy is {}\nAverage SIGFALL accuracy is {}\n---\n'
    print(x.format(test['data_name'],test['model'],test['specificity'],test['sensitivity']))

def balance_data_set(dataset, classifier):
    sigfall_indexes = dataset.index[dataset[classifier] == 0].to_list()
    n_falls = len(dataset) - len(sigfall_indexes)
    drop = len(sigfall_indexes) - n_falls
    random.shuffle(sigfall_indexes)
    drop_indexes = sigfall_indexes[0:drop]
    return dataset.drop(drop_indexes)

# Balance_data_generate_scores() is a seperate function to invoke a new randomly balanced dataset in each iteration.
# If a dataset was pre-balanced then run through generate_scores() it would overfit to that specific dataset 

def balance_data_generate_scores(dataset, classifier, model):
    specificity_scores = []
    sensitivity_scores = []

    for n in range(1000):
        b_dataset = balance_data_set(dataset, classifier)
        cm = build_confusion_matrix(b_dataset, model)
        specificity_scores.append(specificity(cm))
        sensitivity_scores.append(sensitivity(cm))
    
    spec_avg = pd.Series(specificity_scores).mean()
    sens_avg = pd.Series(sensitivity_scores).mean()
    
    return {'specificity': spec_avg, 'sensitivity': sens_avg }

In [11]:
# Load data sets
inc_updrs_sigfall = pd.read_csv('./working_data/normalised_increase_updrs_sigfall.csv')
inc_updrs_sigfall_raw = pd.read_csv('./working_data/normalised_increase_updrs_sigfall_raw.csv')

datasets = [{'data':inc_updrs_sigfall, 'name':'inc_updrs_sigfall'},
            {'data':inc_updrs_sigfall_raw, 'name':'inc_updrs_sigfall_raw'}]

In [12]:
# Define models 
models = [
    LinearDiscriminantAnalysis(solver="svd"),
    LinearDiscriminantAnalysis(solver="lsqr"),
    LinearDiscriminantAnalysis(solver="eigen")
]


In [13]:
# Run tests

tests = []
for m in models:
    for d in datasets:
        tests.append({
            'data_name': d['name'],
            'dataset': d['data'],
            'model': m
        })

for t in tests:
    s = generate_scores(dataset=t['dataset'], model=t['model'])
    t['specificity'] = s['specificity']
    t['sensitivity'] = s['sensitivity']
    summarise_test(t)

For inc_updrs_sigfall, in model LinearDiscriminantAnalysis()
Average NO SIGFALL accuracy is 0.9783344747015499
Average SIGFALL accuracy is 0.12102639323530176
---

For inc_updrs_sigfall_b, in model LinearDiscriminantAnalysis()
Average NO SIGFALL accuracy is 0.7701675882609793
Average SIGFALL accuracy is 0.5414108910159185
---

For inc_updrs_sigfall_raw, in model LinearDiscriminantAnalysis()
Average NO SIGFALL accuracy is 0.9790587162492428
Average SIGFALL accuracy is 0.11451312778477331
---

For inc_updrs_sigfall, in model LinearDiscriminantAnalysis(solver='lsqr')
Average NO SIGFALL accuracy is 0.9787376983900437
Average SIGFALL accuracy is 0.11924606189749122
---

For inc_updrs_sigfall_b, in model LinearDiscriminantAnalysis(solver='lsqr')
Average NO SIGFALL accuracy is 0.7746724080271306
Average SIGFALL accuracy is 0.5406483697831564
---

For inc_updrs_sigfall_raw, in model LinearDiscriminantAnalysis(solver='lsqr')
Average NO SIGFALL accuracy is 0.9782692943459869
Average SIGFALL accu

In [7]:
# See QDA notebook for implementation of 'Balanced Datasets' - not sure its a valid thing to do ATM

{'specificity': 0.9785999321848587, 'sensitivity': 0.1148227988475207}